# SPSSブログ Python実装

## 共通事前処理

In [1]:
# 必要ライブラリのimport
import pandas as pd
import numpy as np

# データフレーム表示用関数
from IPython.display import display

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

## データ読み込み

In [2]:
# データ読み込み
url = 'https://raw.githubusercontent.com/yoichiro0903n/blue/main/sampledatacross2.csv'
df = pd.read_csv(url, index_col='ID')

display(df.head())

,果物･野菜,肉,日用雑貨,缶詰野菜,缶詰肉,冷凍肉,ビール,ワイン,清涼飲料,魚,菓子
ID,,,,,,,,,,,
1,F,T,T,F,F,F,F,F,F,F,T
2,F,T,F,F,F,F,F,F,F,F,T
3,F,F,F,T,F,T,T,F,F,T,F
4,F,F,T,F,F,F,F,T,F,F,F
5,F,F,F,F,F,F,F,F,F,F,F


In [3]:
# 'T'-> True, 'F'-> Falseに変換
df2 = df.replace({'T': True, 'F': False})

# 結果確認
display(df2.head())

,果物･野菜,肉,日用雑貨,缶詰野菜,缶詰肉,冷凍肉,ビール,ワイン,清涼飲料,魚,菓子
ID,,,,,,,,,,,
1,False,True,True,False,False,False,False,False,False,False,True
2,False,True,False,False,False,False,False,False,False,False,True
3,False,False,False,True,False,True,True,False,False,True,False
4,False,False,True,False,False,False,False,True,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False


## 例題1

### 初期設定

In [4]:
#  項目のリスト
I = df2.columns

# 項目数
N = len(I)

# 結果表の初期設定
S = pd.DataFrame(np.zeros((N, N)), 
    columns=I, index=I, dtype='int64')

### 集計処理

In [5]:
# 3重ループで該当要素値をカウントアップ
for i in df2.values:
    # 要素がTrueの項目のみ抽出
    x1 = I[i].tolist()
    
    # 要素数が１以下の場合は何もしない
    if len(x1) <= 1:
        continue
    
    # 項目ペアの第1要素x2をループで取得
    for x2 in x1:
        # x3: x2以外の要素のリスト
        x3 = x1.copy()
        x3.remove(x2)
    
        # x3は更に各要素x4に分解
        for x4 in x3:
        
        # DataFrameの該当要素値をカウントアップ
            S.loc[x2, x4] += 1
        
# 結果確認
display(S)

,果物･野菜,肉,日用雑貨,缶詰野菜,缶詰肉,冷凍肉,ビール,ワイン,清涼飲料,魚,菓子
果物･野菜,0,59,62,86,61,86,89,84,56,145,82
肉,59,0,33,55,41,52,47,49,42,48,54
日用雑貨,62,33,0,44,31,51,45,46,35,56,56
缶詰野菜,86,55,44,0,73,173,167,97,63,89,71
缶詰肉,61,41,31,73,0,75,60,54,42,63,54
冷凍肉,86,52,51,173,75,0,170,71,54,90,66
ビール,89,47,45,167,60,170,0,77,45,85,64
ワイン,84,49,46,97,54,71,77,0,60,78,144
清涼飲料,56,42,35,63,42,54,45,60,0,52,52
魚,145,48,56,89,63,90,85,78,52,0,86


## 例題2

### 関数定義

In [6]:
# 1次元化したインデックスを2次元に戻し
# 該当項目名をタプルで戻す関数

def getitem(x, I):
    N = len(I)
    x1, x2 = (x % N), (x // N)
    return(I[x1], I[x2])

### 項目ペアのリスト作成

In [7]:
# 上位5個の項目ペアを取得

# 結果リスト初期化
cols_list = []

# 行列の項目値を抜き出し1次元化
w1 = S.values.reshape(-1)

# 上位5個のインデックス値取得
w2 = np.argsort(-w1)[:10:2]

# getitem関数を使って項目名ペアのリスト作成
for i in w2:
    x, y = getitem(i, I)
    cols_list.append([x, y])

# 結果確認
for cols in cols_list:
    print(cols)

['缶詰野菜', '冷凍肉']
['冷凍肉', 'ビール']
['ビール', '缶詰野菜']
['果物･野菜', '魚']
['菓子', 'ワイン']


### 該当データの抽出

In [8]:
# 空のデータフレーム作成
target_df = pd.DataFrame()

# 先頭行から順にループ処理
for i in range(len(df2)):
    
    # 要素がTrueの項目のみ抽出
    item = df2.iloc[i]
    
    # 抽出結果を集合化
    x1 = set(I[item.values].tolist())
    for j in cols_list:
        
        # cols_listの各要素も集合化
        x2 = set(j)
        
        # 集合同士の比較で条件を満たしているかチェック
        if x2 <= x1:
            target_df = target_df.append(df.iloc[i])
            break
        # 以下の実装は2重ループで最初のループにcontinueをかけるためのテクニック
        else:
            continue

In [9]:
# 結果確認
display(target_df.head(6))

# 結果件数確認
count = len(target_df)
print(f'結果件数: {count}')

,ビール,ワイン,冷凍肉,日用雑貨,果物･野菜,清涼飲料,缶詰肉,缶詰野菜,肉,菓子,魚
3,T,F,T,F,F,F,F,T,F,F,T
10,F,F,F,F,T,F,F,F,F,F,T
11,F,T,F,T,T,F,F,T,T,F,T
12,F,F,F,F,T,F,F,F,F,F,T
15,F,F,F,T,T,T,F,F,F,F,T
16,T,F,T,F,F,T,F,F,F,F,F


結果件数: 447
